# Post-Hoc Analysis of Rankings and Relevance Judgments

The notebook below examplifies how `ir_axioms` allows the post-hoc analysis of run files and qrels using the passage retrieval task of the TREC Deep Learning track in 2019 and 2020 as example (using a single BM25 run). In this notebook, we calculate the distribution of axiom preferences in rankings and evaluate the consistency of rankings and relevance judgments with retrieval axioms.


## Preparation

Install the `ir_axioms` framework and [PyTerrier](https://github.com/terrier-org/pyterrier). In Google Colab, we do this automatically.


In [ ]:
from sys import modules

if "google.colab" in modules:
    !pip install -q ir_axioms[experiments] python-terrier

We initialize PyTerrier and import all required libraries and load the data from [ir_datasets](https://ir-datasets.com/).


In [ ]:
from pyterrier import started, init

if not started():
    init(tqdm="auto")

## Datasets and Index

Using PyTerrier's `get_dataset()`, we load the MS MARCO passage ranking dataset.


In [ ]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "msmarco-passage"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")
dataset_train: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2019/judged")
dataset_test: Dataset = get_dataset(f"irds:{dataset_name}/trec-dl-2020/judged")

Now define paths where we will store temporary files, datasets, and the search index.


In [ ]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

If the index is not ready yet, now is a good time to create it and index the MS MARCO passages.
(Lean back and relax as this may take a while...)


In [ ]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(dataset.get_corpus_iter(), fields=["text"])

## Baseline Run

We use PyTerrier's `BatchRetrieve` to create a baseline search pipeline for retrieving with BM25 from the index we just created.


In [ ]:
from pyterrier.batchretrieve import BatchRetrieve

bm25 = BatchRetrieve(str(index_dir.absolute()), wmodel="BM25")

## Import Axioms

Here we're listing which axioms we want to use in our experiments.
Because some axioms require API calls or are computationally expensive, we cache all axioms using `ir_axiom`'s tilde operator (`~`).


In [ ]:
from ir_axioms.axiom import (
    ArgUC,
    QTArg,
    QTPArg,
    aSL,
    PROX1,
    PROX2,
    PROX3,
    PROX4,
    PROX5,
    TFC1,
    TFC3,
    RS_TF,
    RS_TF_IDF,
    RS_BM25,
    RS_PL2,
    RS_QL,
    AND,
    LEN_AND,
    M_AND,
    LEN_M_AND,
    DIV,
    LEN_DIV,
    M_TDC,
    LEN_M_TDC,
    STMC1,
    STMC1_f,
    STMC2,
    STMC2_f,
    LNC1,
    TF_LNC,
    LB1,
    REG,
    ANTI_REG,
    REG_f,
    ANTI_REG_f,
    ASPECT_REG,
    ASPECT_REG_f,
)

axioms = [
    ~ArgUC(),
    ~QTArg(),
    ~QTPArg(),
    ~aSL(),
    ~LNC1(),
    ~TF_LNC(),
    ~LB1(),
    ~PROX1(),
    ~PROX2(),
    ~PROX3(),
    ~PROX4(),
    ~PROX5(),
    ~REG(),
    ~REG_f(),
    ~ANTI_REG(),
    ~ANTI_REG_f(),
    ~ASPECT_REG(),
    ~ASPECT_REG_f(),
    ~AND(),
    ~LEN_AND(),
    ~M_AND(),
    ~LEN_M_AND(),
    ~DIV(),
    ~LEN_DIV(),
    ~RS_TF(),
    ~RS_TF_IDF(),
    ~RS_BM25(),
    ~RS_PL2(),
    ~RS_QL(),
    ~TFC1(),
    ~TFC3(),
    ~M_TDC(),
    ~LEN_M_TDC(),
    ~STMC1(),
    ~STMC1_f(),
    ~STMC2(),
    ~STMC2_f(),
]
axiom_names = [axiom.axiom.name for axiom in axioms]

## Axiomatic Experiments

The `AxiomaticExperiment` class provides the entry-point to the post-hoc analysis of rankings and relevance judgments.
We pass it the retrieval pipelines to evaluate, dataset and index, and the axioms to compute preferences for.
With the `depth` parameter you can control how much preferences are being computed.

Computing time roughly scales with `len(retrieval_systems)` x `len(topics)` x `depth` x `depth` x `len(axioms)`.
Additional parameters help optimize computational costs by filtering missing qrels and/or topics.
In our experience, caching helps most with multiple runs for the same benchmark (like TREC runs), because different run's result sets often overlap.


In [ ]:
from ir_axioms.backend.pyterrier.experiment import AxiomaticExperiment

experiment = AxiomaticExperiment(
    retrieval_systems=[bm25],
    topics=dataset_test.get_topics(),
    qrels=dataset_test.get_qrels(),
    index=index_dir,
    dataset=dataset_name,
    axioms=axioms,
    axiom_names=axiom_names,
    depth=10,
    filter_by_qrels=False,
    filter_by_topics=False,
    verbose=True,
    cache_dir=cache_dir,
)

### Calculate All Preferences

With the `preference` member you can compute a `DataFrame` containing all pairwise preferences up to the specified depth.


In [ ]:
experiment.preferences

### Evaluate Inconsistent Document Pairs

Investigate document pairs that are inconsistent with retrieval axioms (and relevance judgments).
These can provide insights to improve the evaluated ranking function.


In [ ]:
experiment.inconsistent_pairs.mean()

### Evaluate Consistency of Runs with Axioms

This evaluation shows how consistent axioms are with the run's original order (ORIG) and the relevance judgments (ORACLE).
High consistency with the ORACLE axiom indicates that for your run, the axiom captures the relevance judgments well.
High consistency with the ORIG axiom indicates that your run adheres to the axiom's constraint well.


In [ ]:
experiment.preference_consistency

### Evaluate Preference Distribution

Calculating the distribution of each axiom's preferences compared to the original ranking order (ORIG) can help identify decisive axioms for your dataset.


In [ ]:
experiment.preference_distribution